# DT Assignment1

# Data Loading

In [9]:
import pandas as pd 
import numpy as np

In [10]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [11]:
pd_data.iloc[:,0]

0           youth
1           youth
2     middle_aged
3          senior
4          senior
5          senior
6     middle_aged
7           youth
8           youth
9          senior
10          youth
11    middle_aged
12    middle_aged
13         senior
Name: age, dtype: object

In [12]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree.export import export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
def get_gini(df, label):
   
    # 타겟변수의 gini 구하기
    target_class = list(df[label].unique())
    
    target_yes = len(df[df[label]==target_class[0]])
    target_no = len(df[df[label]!=target_class[0]])
    p = target_yes/len(df[label])
    q = 1-p
    
    gini = 1 - p**2-q**2
    
    return  gini

In [15]:
target_class = list(pd_data["class_buys_computer"].unique())
target_class

['no', 'yes']

In [16]:
get_gini(pd_data,'class_buys_computer')     # 답과 같이 나옴

0.4591836734693879

In [17]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693879

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [18]:
# 원리
"""
def get_binary_split(df, attribute):
    
    # 변수의 클래스를 pool로 받음
    pool = df[attribute].unique()     
    pool1 = list(map(list,itertools.combinations(pool,1)))
    pool2 = list(map(list,itertools.combinations(pool,2)))
    #nC1과 nC2
    
    result = pool1 + pool2

    return result 
"""

'\ndef get_binary_split(df, attribute):\n    \n    # 변수의 클래스를 pool로 받음\n    pool = df[attribute].unique()     \n    pool1 = list(map(list,itertools.combinations(pool,1)))\n    pool2 = list(map(list,itertools.combinations(pool,2)))\n    #nC1과 nC2\n    \n    result = pool1 + pool2\n\n    return result \n'

In [19]:
import itertools # 변수의 모든 클래스 조합을 얻기 위해 itertools 불러오기

# 클래스가 4개 이상일 때

def get_binary_split(df,attribute):
    
    pool = df[attribute].unique()     # 변수의 클래스를 pool로 받음
    result = list()
    
    for i in range(0,len(pool)+1):
        result.extend(list(map(list,itertools.combinations(pool,i))))
    
    return result[1:-1]


In [20]:
get_binary_split(pd_data, "age")    # 답과 같이 나옴

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [21]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [22]:
# 클래스 4개 이상일 때 
test = np.array(['A', 'B', 'C','D'])
pool = test     # 변수의 클래스를 pool로 받음
result = list()
    
for i in range(0,len(pool)+1):
     result.extend(list(map(list,itertools.combinations(pool,i))))
result[1:-1]

[['A'],
 ['B'],
 ['C'],
 ['D'],
 ['A', 'B'],
 ['A', 'C'],
 ['A', 'D'],
 ['B', 'C'],
 ['B', 'D'],
 ['C', 'D'],
 ['A', 'B', 'C'],
 ['A', 'B', 'D'],
 ['A', 'C', 'D'],
 ['B', 'C', 'D']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [23]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [30]:
def get_attribute_gini_index(df, attribute, label):
    # get_attribute_gini_index(pd_data, "age", "class_buys_computer")
    
    class_values = list(df[attribute].unique())    # ('youth','middle_age','senior')
    key = get_binary_split(df,attribute)
    gini = list()
    
    for i in range(0,len(class_values)):
        D1 = len(df[df[attribute]==class_values[i]])/len(df)
        D2 = 1-D1
        # get_gini도 이용해서 gini 계산
        gini = D1 * get_gini(df[df[attribute]==class_values[i]],label) + D2 * get_gini(df[df[attribute]!=class_values[i]],label) 
        print(gini)
    
    result = dict(zip(key,gini)) 
    
    return result

In [40]:
# 상연님 우수과제를 통해 수정한 함수 식

def get_attribute_gini_index(df, attribute, label):
    # get_attribute_gini_index(pd_data, "age", "class_buys_computer")
    
    # 앞의 함수 get_binary_split 함수를 이용해서 list이용하기 위해서 keys값에 넣어줌
    result = {}     # result 딕셔너리 만듦
    keys = get_binary_split(df,attribute)
    D_len = df[attribute].shape[0]      # 지니계수 구하고 싶은 특정 피쳐의 길이
    
    # 내가 생각 못했던 get_binary_split으로 얻은 리스트들을 다시 슬라이싱 해서 이용하는 방법
    for key in keys:
        t_index = df[attribute].map(lambda x:x in key)      # keys 리스트에 들은 값들을 index로 반환 
        Dj_len = sum(t_index)        # index가 된 값들의 true 계산, sum
        
        # 지니계수 식에 대입해서 구함.
        gini = (Dj_len/D_len)*get_gini(df[t_index],label) + ((D_len-Dj_len)/D_len)*get_gini(df[~t_index],label)     # ~기호를 잘 사용함.
        
        result[tuple(key)] = gini
    return result 


In [41]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428571,
 ('youth', 'middle_aged'): 0.4571428571428571,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [42]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [101]:
pd_data.columns

Index(['age', 'income', 'student', 'credit_rating', 'class_buys_computer'], dtype='object')

In [102]:
##문제1 답안
print(get_binary_split(pd_data, "income"))

[['high'], ['medium'], ['low'], ['high', 'medium'], ['high', 'low'], ['medium', 'low']]


In [105]:
##문제2 답안

target = "class_buys_computer"
result = {}
for i in range(len(pd_data.columns)-1):
    my_dict_total = get_attribute_gini_index(pd_data,pd_data.columns[i],target)     # pd_data의 각 피쳐에 대한 지니값들은 dict생성
    key_min_total = min(my_dict_total.keys(),key = lambda k : my_dict_total[k])     # 각 변수별 가장 작은 지니 값 구함  

    result.update({key_min_total:my_dict_total[key_min_total]})                     # 가장 작은 지니값을 result에 업데이트 시킴

    
print('Min:',result)
    
    

Min: {('middle_aged',): 0.35714285714285715, ('high',): 0.44285714285714284, ('no',): 0.3673469387755103, ('fair',): 0.42857142857142855}


각 변수별로 지니값을 구한 결과 age의 middle_aged가 가장 작은 지니계수 값을 가짐.  
가장 중요한 변수는 age임을 알 수 있음.

In [113]:
##문제3, 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.
## 답안

# 앞서 구한 가장 중요한 변수 age기준으로 middel_aged / youth,senior 으로 나눠서 새로운 데이터 만듦
youth = pd_data[pd_data['age']=='middle_aged']
senior = pd_data[pd_data['age']=='senior']
pd_data2 = pd.concat([youth,senior],axis=0)
pd_data2

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
9,senior,medium,yes,fair,yes
13,senior,medium,no,excellent,no


In [114]:
get_attribute_gini_index(pd_data2,"income","class_buys_computer")

{('high',): 0.3174603174603175,
 ('low',): 0.33333333333333326,
 ('medium',): 0.3444444444444444,
 ('high', 'low'): 0.3444444444444444,
 ('high', 'medium'): 0.33333333333333326,
 ('low', 'medium'): 0.3174603174603175}

In [109]:
# pd_data2에서 다시 가장 작은 값을 구해봄.

target = "class_buys_computer"
result = {}
for i in range(len(pd_data2.columns)-1):
    my_dict_total = get_attribute_gini_index(pd_data2,pd_data2.columns[i],target)     # pd_data의 각 피쳐에 대한 지니값들은 dict생성
    key_min_total = min(my_dict_total.keys(),key = lambda k : my_dict_total[k])     # 각 변수별 가장 작은 지니 값 구함  

    result.update({key_min_total:my_dict_total[key_min_total]})                     # 가장 작은 지니값을 result에 업데이트 시킴

    
print('Min:',result)

Min: {('middle_aged',): 0.26666666666666666, ('high',): 0.3174603174603175, ('no',): 0.3444444444444444, ('fair',): 0.2222222222222222}


age 다음으로 중요한 변수는 'fair'가 들어있는 credit rating으로 우수답안과 답이 다릅니다.  
이유가 뭘까요?  